In [ ]:
### This file does a RZ gate (1 parameter, 1 qubit) with X,Y control
using QuantumCollocation
using NamedTrajectories
using TrajectoryIndexingUtils

In [ ]:
const Units = 1e9
const MHz = 1e6 / Units
const GHz = 1e9 / Units
const ns = 1e-9 * Units
const μs = 1e-6 * Units
;

In [ ]:
# Operators
const Paulis = Dict(
    "I" => Matrix{ComplexF64}([1 0; 0 1]),
    "X" => Matrix{ComplexF64}([0 1; 1 0]),
    "Y" => Matrix{ComplexF64}([0 im; -im 0]),
    "Z" => Matrix{ComplexF64}([1 0; 0 -1]),
)
const a = [0 1; 0 0]
const ad = transpose(a);

H_drift = []



function indexedOp(op, i)
    init = [Paulis["I"] for j in 1:4]
    init[i] = op
    return reduce(kron,init)
end

excitation(theta)   = exp(theta * ( (indexedOp(ad,1) *indexedOp(ad,2) * indexedOp(a,3) * indexedOp(a,4) ) - 
        ((indexedOp(ad,3) *indexedOp(ad,4) * indexedOp(a,1) * indexedOp(a,2)))))


X_controls = [indexedOp(Paulis["X"], i) for i in 1:2]
Y_controls = [indexedOp(Paulis["Y"], i) for i in 1:2]
XY_controls = reduce(vcat,[[(indexedOp(Paulis["X"],i)*indexedOp(Paulis["X"],j)+indexedOp(Paulis["Y"],i)*indexedOp(Paulis["Y"],j))/2 for j in 1:4] for i in 1:4])

H_drives = reduce(vcat,[X_controls,Y_controls,XY_controls])

system = QuantumSystem(H_drives);
t_f = 10 * ns
n_steps = 51
times = range(0, t_f, n_steps)  # Alternative: collect(0:Δt:t_f)
Δt = times[2] - times[1]
n_controls=length(H_drives)
n_qubits=4;

In [ ]:
### Generate Initial Trajectories 
PICO_max_iter = 100

# Shape the cost function with weights on states and controls
Q = 1000.
R =1e-2
R_b=200.0
# Add control bounds
a_bound= 1.0
dda_bound = 1.0

ops = Options()
ops.print_info_string = "yes"
ops.recalc_y = "yes"
ops.recalc_y_feas_tol = 1.0 ##down
ops.print_level = 2
p = UnitarySmoothPulseProblem(
                system,
                excitation(pi),
                n_steps,
                Δt;
                a_bound=a_bound,
                dda_bound=dda_bound,
                Q=1000.0,
                R=1e-2,
                verbose=true,
                hessian_approximation=true,
                pade_order=10,
                free_time=true,
                timesteps_all_equal=true,
                max_iter=PICO_max_iter,
                ipopt_options=ops,
            )
min_time_problem  = UnitaryMinimumTimeProblem(p; final_fidelity=1-1e-5,max_iter=100)
solve!(min_time_problem)

In [ ]:
Δt = ceil(1.2 * min_time_problem.trajectory[:Δt][1]*100)/100

In [ ]:
N = 11


trajectory_list,glued_trajectory_list = Interpolator1D(
    excitation,
    system,
    n_steps,
    N,
    Δt;
    Q=Q,
    R=R,
    R_b=R_b,
    a_bound=a_bound,
    dda_bound=dda_bound,
    ipopt_options=ops,
    max_iter=PICO_max_iter
);

In [ ]:
DATA = sample1D(trajectory_list,Δt,n_qubits,system,2500,N,excitation)
findmax(DATA)

In [ ]:
using CairoMakie
using Colors
using Printf
f = Figure()
ax = Axis(f[1, 1],
    title = "Infidelity Log Plot",
    xlabel = "θ",
    ylabel = "Infidelity"
)
lines!(ax, range(0,2*pi,convert(Int64,2500)), log10.(convert(Array{Float64,1},DATA)), label  = "Linear",color = :blue)


f[1, 2] = Legend(f, ax, "Infidelity Data", framevisible = false)
f
     

In [ ]:
DATA = simple_sample1D(glued_trajectory_list,Δt,n_qubits,system,2500,N,excitation)
findmax(DATA)

In [ ]:
using CairoMakie
using Colors
using Printf
f = Figure()
ax = Axis(f[1, 1],
    title = "Infidelity Log Plot",
    xlabel = "θ",
    ylabel = "Infidelity"
)
lines!(ax, range(0,2*pi,convert(Int64,2500)), log10.(convert(Array{Float64,1},DATA)), label  = "Linear",color = :blue)

f[1, 2] = Legend(f, ax, "Infidelity Data", framevisible = false)
f
     

In [ ]:
data,d_data,dd_data = parse1D(trajectory_list,n_controls,N);

In [ ]:
verify1D([t[:dda] for t in glued_trajectory_list],n_qubits,system,Δt,n_steps,N,excitation)

In [ ]:
verify1D(dd_data,n_qubits,system,Δt,n_steps,N,excitation)

In [ ]:
export_csv1D([t[:dda] for t in glued_trajectory_list],n_controls,N,"XYControlAdapt4_accels.csv")